In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from lxml import html
import pandas as pd

# Data from the United States

## COVID-19 Cases and Deaths

The _New York Times_ gathers number of confirmed cases and deaths from the state governments on the daily basis, and releases them on their github. The dataset is avilable at [this link](https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-states.csv).

In [2]:
us_cases = pd.read_csv("https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-states.csv")
us_cases.head(5)

,date,state,fips,cases,deaths
0,2020-01-21,Washington,53,1,0
1,2020-01-22,Washington,53,1,0
2,2020-01-23,Washington,53,1,0
3,2020-01-24,Illinois,17,1,0
4,2020-01-24,Washington,53,1,0


The `fips` and `deaths` variables in this dataset will not be used. Therefore, we are removing it.

In [3]:
us_cases = us_cases.drop(['fips'], axis=1)
us_cases.head(5)

,date,state,cases,deaths
0,2020-01-21,Washington,1,0
1,2020-01-22,Washington,1,0
2,2020-01-23,Washington,1,0
3,2020-01-24,Illinois,1,0
4,2020-01-24,Washington,1,0


To measure the effectiveness of the disease control, we will look at the percentage increase in cases 


*   Between 15 MAR 2020 to 14 APR 2020, and
*   Between 15 APR 2020 to 14 MAY 2020,



In [0]:
march_cases = us_cases[us_cases['date'] == '2020-03-15']
march_cases = march_cases.drop(['date'], axis=1)
april_cases = us_cases[us_cases['date'] == '2020-04-15']
april_cases = april_cases.drop(['date'], axis=1)
may_cases = us_cases[us_cases['date'] == '2020-05-15']
may_cases = may_cases.drop(['date'], axis=1)

In [5]:
temp_1 = pd.merge(march_cases, april_cases, on = "state", suffixes=["_MAR", "_APR"])
us_cases_aggregated = pd.merge(temp_1, may_cases, on = "state")
del temp_1, march_cases, april_cases, may_cases
us_cases_aggregated.columns = ['state', 'cases_MAR', 'deaths_MAR', 'cases_APR', 'deaths_APR', 'cases_MAY', 'deaths_MAY']
us_cases_aggregated.head(5)

,state,cases_MAR,deaths_MAR,cases_APR,deaths_APR,cases_MAY,deaths_MAY
0,Alabama,23,0,4241,123,11373,483
1,Alaska,1,0,291,7,388,8
2,Arizona,13,0,3962,142,13169,651
3,Arkansas,16,0,1599,34,4463,98
4,California,478,6,27107,885,77015,3192


In [0]:
us_cases_aggregated["cases_MAR_to_APR"] = (us_cases_aggregated["cases_APR"] - us_cases_aggregated["cases_MAR"]) / us_cases_aggregated["cases_MAR"] * 100
us_cases_aggregated["cases_APR_to_MAY"] = (us_cases_aggregated["cases_MAY"] - us_cases_aggregated["cases_APR"]) / us_cases_aggregated["cases_APR"] * 100

In [7]:
us_cases_aggregated = us_cases_aggregated.drop(['cases_MAR', 'deaths_MAR', 'cases_APR', 'deaths_APR', 'deaths_MAY'], axis=1)
us_cases_aggregated.head(5)

,state,cases_MAY,cases_MAR_to_APR,cases_APR_to_MAY
0,Alabama,11373,18339.130435,168.167885
1,Alaska,388,29000.000000,33.333333
2,Arizona,13169,30376.923077,232.382635
3,Arkansas,4463,9893.750000,179.111945
4,California,77015,5570.920502,184.114804


## US Population Data

Another measurement we wish to use is the percentage of the population that was confirmed COVID-19. The population data is necessary here. United States Census Bureau conducts censuses for population data, and releases an estimate for the US population by state every year. The latest data is available at [this link](https://www.census.gov/data/tables/time-series/demo/popest/2010s-state-total.html#par_textimage).

In [8]:
us_pop = pd.read_csv('http://www2.census.gov/programs-surveys/popest/datasets/2010-2019/national/totals/nst-est2019-alldata.csv')
us_pop = us_pop[['NAME', "POPESTIMATE2019"]]
us_pop.columns = ['state', 'Population']
us_pop.head(5)

,state,Population
0,United States,328239523
1,Northeast Region,55982803
2,Midwest Region,68329004
3,South Region,125580448
4,West Region,78347268


In [0]:
us_cases_aggregated = pd.merge(us_cases_aggregated, us_pop, on = 'state')

In [10]:
us_cases_aggregated['Cases per 1000 in population'] = us_cases_aggregated["cases_MAY"] / us_cases_aggregated["Population"] * 1000
us_cases_aggregated = us_cases_aggregated.drop(['cases_MAY', 'Population'], axis = 1)
us_cases_aggregated.head(5)

,state,cases_MAR_to_APR,cases_APR_to_MAY,Cases per 1000 in population
0,Alabama,18339.130435,168.167885,2.319513
1,Alaska,29000.000000,33.333333,0.530384
2,Arizona,30376.923077,232.382635,1.809247
3,Arkansas,9893.750000,179.111945,1.478890
4,California,5570.920502,184.114804,1.949144


## US Unemployment Data 

When governments take actions to prevent the spread of COVID-19, there may be negative impacts of the welfare of the labours. One data point we often look at is the unemployment. 

The US Bureau of Labour Statistics releases the labour data by state on the monthly basis, and they are available at [this webpage](https://www.bls.gov/news.release/laus.t01.htm#). Unfortunatly, they did not provide downloadable data and/or API. Python Spider, in this case, will help us fetch the data from their webpage.

In [11]:
import requests

url = 'https://www.bls.gov/news.release/laus.t01.htm#'
page = requests.get(url)
print(page.text)

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN"  "http://www.w3.org/TR/html4/loose.dtd"> 
<html lang="en"> 
<HEAD>
<TITLE> Table 1.  Civilian labor force and unemployment by state and selected area, seasonally adjusted </TITLE>

	<meta name="agency" content="BLS" />
	<meta name="audience" content="general" />	<meta http-equiv="expires" content="-1">
	<meta http-equiv="Pragma" content="no-cache">

    <!--[if lt IE 9]>
		<link href="/stylesheets/ie8-styles.css" media="all" rel="Stylesheet" type="text/css"> 
	<![endif]-->
	<!--[if lt IE 8]>
		<link href="/stylesheets/ie7-styles.css" media="all" rel="Stylesheet" type="text/css"> 
	<![endif]-->
	<!--[if lt IE 7]>
		<link href="/stylesheets/ie6-styles.css" media="all" rel="Stylesheet" type="text/css"> 
		<style type="text/css">
		body {
			behavior:url(/stylesheets/csshover2.htc);
		}
		</style>			
	<![endif]-->
	<!--[if IE]>
		<style type="text/css">
		body {
			font-size: expression(1 / (screen.deviceXDPI / 96) * 84 + '%');

In [12]:
tree = html.fromstring(page.content)
headers = ['state', 'Apr. 2019', 'Feb. 2020', 'Mar. 2020', 'Apr. 2020']
provinces = np.array([i.text for i in tree.xpath('//table//tbody//tr//th//p')]).reshape((59, 1))
entries = np.array(tree.xpath('//table//tbody//tr//td//span[@class="datavalue"]//text()')).reshape((59, 12))
entries = np.hstack([provinces, entries[:,4:]])
result = pd.DataFrame({headers[i]: entries[:,i] for i in range(len(headers))})
result.head(5)

,state,Apr. 2019,Feb. 2020,Mar. 2020,Apr. 2020
0,Alabama,"71,185","59,655","67,004","283,787"
1,Alaska,"21,355","20,095","18,015","43,683"
2,Arizona,"169,458","161,238","219,151","445,461"
3,Arkansas,"48,175","47,802","69,728","133,962"
4,California,"806,461","759,328","1,052,163","2,885,334"


To measure the impact on the labour market caused by the government policies, we will look at

* The change in Unemployment Rate from MAR 2020 to APR 2020, and
* The change in Unemployment Rate from APR 2019 to APR 2020.

These two changes are called MoM growth and YoY growth. Before proceeding, we see that Puerto Rico have data missing. So we will simply delete it from the dataset.

In [0]:
result = result.drop([58], axis=0)

In [0]:
for i in ['Apr. 2019', 'Feb. 2020', 'Mar. 2020', 'Apr. 2020']:
    result[i] = result[i].str.replace(',', '').astype(float)

In [15]:
result["YoY Growth"] = (result["Apr. 2020"].astype(np.int64) - result["Apr. 2019"].astype(np.int64)) / result["Apr. 2019"].astype(np.float64) * 100 
result["MoM Growth"] = (result["Apr. 2020"].astype(np.int64) - result["Mar. 2020"].astype(np.int64)) / result["Mar. 2020"].astype(np.float64) * 100
result = result.drop(['Apr. 2019', 'Feb. 2020', 'Mar. 2020', 'Apr. 2020'], axis = 1)
result.head(5)

,state,YoY Growth,MoM Growth
0,Alabama,298.661235,323.537401
1,Alaska,104.556310,142.481266
2,Arizona,162.873986,103.266697
3,Arkansas,178.073690,92.120812
4,California,257.777251,174.228803


In [0]:
us_cases_aggregated = pd.merge(us_cases_aggregated, result, on = 'state')
del result

In [17]:
us_cases_aggregated.head(5)

,state,cases_MAR_to_APR,cases_APR_to_MAY,Cases per 1000 in population,YoY Growth,MoM Growth
0,Alabama,18339.130435,168.167885,2.319513,298.661235,323.537401
1,Alaska,29000.000000,33.333333,0.530384,104.556310,142.481266
2,Arizona,30376.923077,232.382635,1.809247,162.873986,103.266697
3,Arkansas,9893.750000,179.111945,1.478890,178.073690,92.120812
4,California,5570.920502,184.114804,1.949144,257.777251,174.228803


## Indicators for the Policies Carried out by the State Government

Our member team member Jinghan Wang searched tirelessly for policies carried out by the state governments. We used indicators to show if a state carried out a specific policy.

In [18]:
us_policy = pd.read_csv("./us_policy.csv")
us_policy.head(5)

,State/Province,Declared state of emergency,Ordered Closure of K-12 schools,Closed daycares,Banned visits to long-term care homes,Closed non-essential businesses,Closed restaurants except take out,Closed cannabis and liquor stores,Closed gyms,Closed movie theaters,Froze evictions,Ordered freezing utility shut offs,Public Face Mask Mandate,Employee Face Mask Mandate,Travel Restriction,Large Gathering ban,Employee Screening
0,Alabama,1,1,1,1,1,1,1,1,1,0,0,0,1,0,1,0
1,Alaska,1,1,0,0,1,1,1,1,1,1,1,0,1,1,1,0
2,Arizona,1,1,0,0,1,1,1,1,1,0,0,0,1,1,1,0
3,Arkansas,1,1,0,1,0,1,1,1,0,0,1,0,1,1,1,1
4,California,1,0,0,0,1,1,1,1,1,1,1,0,1,0,1,1


In [0]:
# Change the column name from 'state' to 'Province/Territory' to be consistent with 
# the response variable.
us_cases_aggregated.columns = ['State/Province', 'cases_MAR_to_APR', 'cases_APR_to_MAY',
                               'percentage_infected_MAY', 'YoY Growth', 'MoM Growth']

In [20]:
us_cases_aggregated = pd.merge(us_cases_aggregated, us_policy, on = "State/Province")
us_cases_aggregated.head(5)

,State/Province,cases_MAR_to_APR,cases_APR_to_MAY,percentage_infected_MAY,YoY Growth,MoM Growth,Declared state of emergency,Ordered Closure of K-12 schools,Closed daycares,Banned visits to long-term care homes,Closed non-essential businesses,Closed restaurants except take out,Closed cannabis and liquor stores,Closed gyms,Closed movie theaters,Froze evictions,Ordered freezing utility shut offs,Public Face Mask Mandate,Employee Face Mask Mandate,Travel Restriction,Large Gathering ban,Employee Screening
0,Alabama,18339.130435,168.167885,2.319513,298.661235,323.537401,1,1,1,1,1,1,1,1,1,0,0,0,1,0,1,0
1,Alaska,29000.000000,33.333333,0.530384,104.556310,142.481266,1,1,0,0,1,1,1,1,1,1,1,0,1,1,1,0
2,Arizona,30376.923077,232.382635,1.809247,162.873986,103.266697,1,1,0,0,1,1,1,1,1,0,0,0,1,1,1,0
3,Arkansas,9893.750000,179.111945,1.478890,178.073690,92.120812,1,1,0,1,0,1,1,1,0,0,1,0,1,1,1,1
4,California,5570.920502,184.114804,1.949144,257.777251,174.228803,1,0,0,0,1,1,1,1,1,1,1,0,1,0,1,1


In [0]:
us_cases_aggregated.to_csv('us_data.csv', index = False)

# Data from Canada

## COVID-19 Cases and Deaths

The following dataset is relased by the Government of Canada. It includes the number of cases and deaths by day by province. This dataset is available at [this link](https://health-infobase.canada.ca/src/data/covidLive/covid19.csv).

In [22]:
can_cases = pd.read_csv("https://health-infobase.canada.ca/src/data/covidLive/covid19.csv")
can_cases.head(5)

,pruid,prname,prnameFR,date,numconf,numprob,numdeaths,numtotal,numtested,numrecover,percentrecover,ratetested,numtoday,percentoday,ratetotal,ratedeaths,deathstoday,percentdeath,testedtoday,recoveredtoday,percentactive
0,35,Ontario,Ontario,31-01-2020,3,0,0.0,3,NaN,NaN,NaN,NaN,3,300.0,0.02,0.0,0.0,0.0,NaN,NaN,100.0
1,59,British Columbia,Colombie-Britannique,31-01-2020,1,0,0.0,1,NaN,NaN,NaN,NaN,1,100.0,0.02,0.0,0.0,0.0,NaN,NaN,100.0
2,1,Canada,Canada,31-01-2020,4,0,0.0,4,NaN,NaN,NaN,NaN,4,400.0,0.01,0.0,0.0,0.0,NaN,NaN,100.0
3,35,Ontario,Ontario,08-02-2020,3,0,0.0,3,NaN,NaN,NaN,NaN,0,0.0,0.02,0.0,0.0,0.0,NaN,NaN,100.0
4,59,British Columbia,Colombie-Britannique,08-02-2020,4,0,0.0,4,NaN,NaN,NaN,NaN,3,300.0,0.08,0.0,0.0,0.0,NaN,NaN,100.0


First, let's remove sum columns that looks too intriguing.

In [23]:
can_cases = can_cases[['prname', 'date', 'numconf', 'numdeaths']]
can_cases.head(5)

,prname,date,numconf,numdeaths
0,Ontario,31-01-2020,3,0.0
1,British Columbia,31-01-2020,1,0.0
2,Canada,31-01-2020,4,0.0
3,Ontario,08-02-2020,3,0.0
4,British Columbia,08-02-2020,4,0.0


In [0]:
can_cases.columns = ['state', 'date', 'cases', 'deaths']
march_cases = can_cases[can_cases['date'] == '15-03-2020']
march_cases = march_cases.drop(['date'], axis=1)
april_cases = can_cases[can_cases['date'] == '15-04-2020']
april_cases = april_cases.drop(['date'], axis=1)
may_cases = can_cases[can_cases['date'] == '15-05-2020']
may_cases = may_cases.drop(['date'], axis=1)

In [25]:
temp_1 = pd.merge(march_cases, april_cases, on = "state", suffixes=["_MAR", "_APR"])
can_cases_aggregated = pd.merge(temp_1, may_cases, on = "state")
del temp_1, march_cases, april_cases, may_cases
can_cases_aggregated.columns = ['state', 'cases_MAR', 'deaths_MAR', 'cases_APR', 'deaths_APR', 'cases_MAY', 'deaths_MAY']
can_cases_aggregated = can_cases_aggregated[can_cases_aggregated["cases_MAR"] > 0]
can_cases_aggregated = can_cases_aggregated.drop([13, 14], axis=0)
can_cases_aggregated

,state,cases_MAR,deaths_MAR,cases_APR,deaths_APR,cases_MAY,deaths_MAY
0,British Columbia,73,1.0,1561,75.0,2407,140.0
1,Alberta,39,0.0,1996,48.0,6515,125.0
3,Manitoba,4,0.0,231,5.0,278,7.0
4,Ontario,103,1.0,8447,385.0,21922,1825.0
5,Quebec,24,0.0,14860,487.0,41420,3401.0
7,New Brunswick,1,0.0,117,0.0,120,0.0
9,Prince Edward Island,1,0.0,26,0.0,27,0.0


In [0]:
can_cases_aggregated["cases_MAR_to_APR"] = (can_cases_aggregated["cases_APR"] - can_cases_aggregated["cases_MAR"]) / can_cases_aggregated["cases_MAR"] * 100
can_cases_aggregated["cases_APR_to_MAY"] = (can_cases_aggregated["cases_MAY"] - can_cases_aggregated["cases_APR"]) / can_cases_aggregated["cases_APR"] * 100

In [0]:
can_cases_aggregated = can_cases_aggregated.drop(['cases_MAR', 'deaths_MAR', 'cases_APR', 'deaths_APR', 'deaths_MAY'], axis=1)
can_cases_aggregated.rename(columns = {'state':'State/Province'}, inplace= True)

In [28]:
can_cases_aggregated

,State/Province,cases_MAY,cases_MAR_to_APR,cases_APR_to_MAY
0,British Columbia,2407,2038.356164,54.196028
1,Alberta,6515,5017.948718,226.402806
3,Manitoba,278,5675.000000,20.346320
4,Ontario,21922,8100.970874,159.524091
5,Quebec,41420,61816.666667,178.734859
7,New Brunswick,120,11600.000000,2.564103
9,Prince Edward Island,27,2500.000000,3.846154


##Canadian Population Data
The following dataset includes Canadian Population statistics as well as Canadian Labour statistics. This dataset is released by the government of Canada, and it is available at [this link](https://www150.statcan.gc.ca/n1/en/tbl/csv/14100287-eng.zip).

In [29]:
! wget https://www150.statcan.gc.ca/n1/en/tbl/csv/14100287-eng.zip
! unzip 14100287-eng.zip

--2020-06-14 23:10:43--  https://www150.statcan.gc.ca/n1/en/tbl/csv/14100287-eng.zip
Resolving www150.statcan.gc.ca (www150.statcan.gc.ca)... 205.193.226.160
Connecting to www150.statcan.gc.ca (www150.statcan.gc.ca)|205.193.226.160|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 52441025 (50M) [application/zip]
Saving to: ‘14100287-eng.zip.1’

14100287-eng.zip.1   45%[========>           ]  22.97M   644KB/s    eta 43s    ^C
Archive:  14100287-eng.zip
replace 14100287.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [30]:
can_population_data = pd.read_csv("14100287.csv")
can_population_data["REF_DATE"] = pd.to_datetime(can_population_data["REF_DATE"])
import datetime
#date = ['2020-02-01','2020-03-01', '2020-04-01', '2019-04-01']
can_population_data = can_population_data[can_population_data["GEO"] != "Canada"]
can_population_data = can_population_data[can_population_data["Data type"] == 'Seasonally adjusted']
can_population_data = can_population_data[can_population_data["Sex"] == 'Both sexes']
can_population_data = can_population_data[can_population_data["Age group"] == '15 years and over']
can_population_data = can_population_data[can_population_data["Statistics"] == 'Estimate']
can_population = can_population_data[can_population_data["Labour force characteristics"] == 'Population']
can_population = can_population.drop(["Data type", "Sex","DGUID","Labour force characteristics", "Age group","Statistics", "UOM","UOM_ID","SCALAR_FACTOR","SCALAR_ID","VECTOR","COORDINATE","STATUS","SYMBOL","TERMINATED","DECIMALS"], axis=1)
can_population

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,REF_DATE,GEO,VALUE
1041,1976-01-01,Newfoundland and Labrador,367.7
1836,1976-01-01,Prince Edward Island,82.1
2631,1976-01-01,Nova Scotia,583.1
3426,1976-01-01,New Brunswick,476.8
4221,1976-01-01,Quebec,4696.3
...,...,...,...
4788228,2020-05-01,Ontario,12307.2
4789023,2020-05-01,Manitoba,1046.2
4789818,2020-05-01,Saskatchewan,893.9
4790613,2020-05-01,Alberta,3571.9


In [31]:
can_population = can_population[["GEO", "VALUE"]][can_population["REF_DATE"] == "2020-03-01"]
can_population.columns = ["State/Province", "Population"]
can_population

,State/Province,Population
4766271,Newfoundland and Labrador,439.8
4767066,Prince Edward Island,130.7
4767861,Nova Scotia,814.3
4768656,New Brunswick,635.3
4769451,Quebec,7102.4
4770246,Ontario,12289.1
4771041,Manitoba,1046.3
4771836,Saskatchewan,894.7
4772631,Alberta,3566.7
4773426,British Columbia,4154.4


In [0]:
can_cases_aggregated = pd.merge(can_cases_aggregated, can_population, on = 'State/Province')

In [33]:
can_cases_aggregated['Cases per 1000 in population'] = can_cases_aggregated["cases_MAY"] / can_cases_aggregated["Population"]
can_cases_aggregated = can_cases_aggregated.drop(['cases_MAY', 'Population'], axis = 1)
can_cases_aggregated.head(5)

,State/Province,cases_MAR_to_APR,cases_APR_to_MAY,Cases per 1000 in population
0,British Columbia,2038.356164,54.196028,0.579386
1,Alberta,5017.948718,226.402806,1.826618
2,Manitoba,5675.000000,20.346320,0.265698
3,Ontario,8100.970874,159.524091,1.783857
4,Quebec,61816.666667,178.734859,5.831831


##Canadian Unemployment Data
The Canadian population dataset that we have used above includes Canadian labour statistics as well. This dataset is released by the government of Canada, and it is available at [this link](https://www150.statcan.gc.ca/n1/en/tbl/csv/14100287-eng.zip).

In [34]:
can_labour_data1 = can_population_data[can_population_data["Labour force characteristics"] == 'Unemployment']
can_labour_data1 = can_labour_data1.drop(["Data type", "Sex","DGUID","Labour force characteristics", "Age group","Statistics", "UOM","UOM_ID","SCALAR_FACTOR","SCALAR_ID","VECTOR","COORDINATE","STATUS","SYMBOL","TERMINATED","DECIMALS"], axis=1)
can_labour_data1.head(5)

,REF_DATE,GEO,VALUE
1452,1976-01-01,Newfoundland and Labrador,22.7
2247,1976-01-01,Prince Edward Island,4.7
3042,1976-01-01,Nova Scotia,25.9
3837,1976-01-01,New Brunswick,29.1
4632,1976-01-01,Quebec,225.3


In [0]:
can_labour_mar20 = can_labour_data1[["GEO", "VALUE"]][can_labour_data1["REF_DATE"] == "2020-03-01"]
can_labour_apr20 = can_labour_data1[["GEO", "VALUE"]][can_labour_data1["REF_DATE"] == "2020-04-01"]
can_labour_apr19 = can_labour_data1[["GEO", "VALUE"]][can_labour_data1["REF_DATE"] == "2019-04-01"]

In [36]:
temp_1 = pd.merge(can_labour_apr19, can_labour_mar20, on="GEO", suffixes=["_APR_19", "_MAR_20"])
canada_labour = pd.merge(temp_1, can_labour_apr20, on = "GEO")
print(canada_labour.columns)
canada_labour.head(5)

Index(['GEO', 'VALUE_APR_19', 'VALUE_MAR_20', 'VALUE'], dtype='object')


,GEO,VALUE_APR_19,VALUE_MAR_20,VALUE
0,Newfoundland and Labrador,30.6,28.9,35.8
1,Prince Edward Island,7.3,7.4,8.4
2,Nova Scotia,34.7,44.1,54.0
3,New Brunswick,30.9,33.3,47.2
4,Quebec,225.0,362.8,729.4


In [37]:
canada_labour.columns = ['GEO','Apr. 2019', 'Mar. 2020', 'Apr. 2020']
canada_labour.head(5)

,GEO,Apr. 2019,Mar. 2020,Apr. 2020
0,Newfoundland and Labrador,30.6,28.9,35.8
1,Prince Edward Island,7.3,7.4,8.4
2,Nova Scotia,34.7,44.1,54.0
3,New Brunswick,30.9,33.3,47.2
4,Quebec,225.0,362.8,729.4


To measure the impact on the labour market caused by the government policies, we will look at

* The change in Unemployment from MAR 2020 to APR 2020, and
* The change in Unemployment from APR 2019 to APR 2020.

These two changes are called MoM growth and YoY growth. 

In [0]:
canada_labour["YoY Growth"] = (canada_labour["Apr. 2020"].astype(np.int64) - canada_labour["Apr. 2019"].astype(np.int64)) / canada_labour["Apr. 2019"].astype(np.float64) * 100 
canada_labour["MoM Growth"] = (canada_labour["Apr. 2020"].astype(np.int64) - canada_labour["Mar. 2020"].astype(np.int64)) / canada_labour["Mar. 2020"].astype(np.float64) * 100
canada_labour = canada_labour.drop(['Apr. 2019', 'Mar. 2020', 'Apr. 2020'], axis = 1)
canada_labour.columns=["State/Province","YoY Growth","MoM Growth"]

In [39]:
canada_labour

,State/Province,YoY Growth,MoM Growth
0,Newfoundland and Labrador,16.339869,24.221453
1,Prince Edward Island,13.698630,13.513514
2,Nova Scotia,57.636888,22.675737
3,New Brunswick,55.016181,42.042042
4,Quebec,224.000000,101.157663
5,Ontario,77.005808,40.471311
6,Manitoba,107.042254,68.337130
7,Saskatchewan,92.261905,47.945205
8,Alberta,80.712166,44.655582
9,British Columbia,125.303153,49.276915


In [0]:
can_cases_aggregated = pd.merge(can_cases_aggregated, canada_labour, on = 'State/Province')
del canada_labour

In [41]:
can_cases_aggregated.head(5)

,State/Province,cases_MAR_to_APR,cases_APR_to_MAY,Cases per 1000 in population,YoY Growth,MoM Growth
0,British Columbia,2038.356164,54.196028,0.579386,125.303153,49.276915
1,Alberta,5017.948718,226.402806,1.826618,80.712166,44.655582
2,Manitoba,5675.000000,20.346320,0.265698,107.042254,68.337130
3,Ontario,8100.970874,159.524091,1.783857,77.005808,40.471311
4,Quebec,61816.666667,178.734859,5.831831,224.000000,101.157663


## Indicators for the Policies Carried out by the Provincial Government

In [42]:
can_policy = pd.read_csv("./canada_policy.csv")
can_policy.rename(columns={'Province/Territory':'State/Province'}, 
                 inplace=True)
can_policy.head(5)

,State/Province,Declared state of emergency,Ordered Closure of K-12 schools,Closed daycares,Banned visits to long-term care homes,Closed non-essential businesses,Closed restaurants except take out,Closed cannabis and liquor stores,Closed gyms,Closed movie theaters,Froze evictions,Ordered freezing utility shut offs,Public Face Mask Mandate,Employee Face Mask Mandate,Travel Restriction,Large Gathering ban,Employee Screening
0,Newfoundland and Labrador,1,1,1,1,1,1,1,1,1,1,1,0,0,1,1,1
1,Prince Edward Island,1,1,1,1,1,1,1,1,1,1,0,0,0,1,1,1
2,Nova Scotia,1,1,1,1,0,1,0,1,0,1,1,0,0,1,1,0
3,New Brunswick,1,1,1,0,1,1,0,1,1,1,1,1,0,1,1,0
4,Quebec,1,1,1,1,1,1,0,1,1,1,1,0,0,0,1,0


In [43]:
can_cases_aggregated = pd.merge(can_cases_aggregated, can_policy, on = "State/Province")
can_cases_aggregated.head(5)

,State/Province,cases_MAR_to_APR,cases_APR_to_MAY,Cases per 1000 in population,YoY Growth,MoM Growth,Declared state of emergency,Ordered Closure of K-12 schools,Closed daycares,Banned visits to long-term care homes,Closed non-essential businesses,Closed restaurants except take out,Closed cannabis and liquor stores,Closed gyms,Closed movie theaters,Froze evictions,Ordered freezing utility shut offs,Public Face Mask Mandate,Employee Face Mask Mandate,Travel Restriction,Large Gathering ban,Employee Screening
0,British Columbia,2038.356164,54.196028,0.579386,125.303153,49.276915,1,1,0,0,0,1,0,0,0,1,1,0,0,1,1,1
1,Alberta,5017.948718,226.402806,1.826618,80.712166,44.655582,1,1,1,1,1,1,0,1,1,1,1,0,0,1,1,0
2,Manitoba,5675.000000,20.346320,0.265698,107.042254,68.337130,1,1,1,1,1,1,0,1,1,1,1,0,0,1,1,1
3,Ontario,8100.970874,159.524091,1.783857,77.005808,40.471311,1,1,1,1,1,1,1,1,1,1,1,0,0,0,1,0
4,Quebec,61816.666667,178.734859,5.831831,224.000000,101.157663,1,1,1,1,1,1,0,1,1,1,1,0,0,0,1,0


In [0]:
can_cases_aggregated.to_csv('can_data.csv', index = False)

# Merge the data from the US and Canada

In [45]:
NA_data = pd.concat([us_cases_aggregated, can_cases_aggregated])
NA_data

,State/Province,cases_MAR_to_APR,cases_APR_to_MAY,percentage_infected_MAY,YoY Growth,MoM Growth,Declared state of emergency,Ordered Closure of K-12 schools,Closed daycares,Banned visits to long-term care homes,Closed non-essential businesses,Closed restaurants except take out,Closed cannabis and liquor stores,Closed gyms,Closed movie theaters,Froze evictions,Ordered freezing utility shut offs,Public Face Mask Mandate,Employee Face Mask Mandate,Travel Restriction,Large Gathering ban,Employee Screening,Cases per 1000 in population
0,Alabama,18339.130435,168.167885,2.319513,298.661235,323.537401,1,1,1,1,1,1,1,1,1,0,0,0,1,0,1,0,NaN
1,Alaska,29000.000000,33.333333,0.530384,104.556310,142.481266,1,1,0,0,1,1,1,1,1,1,1,0,1,1,1,0,NaN
2,Arizona,30376.923077,232.382635,1.809247,162.873986,103.266697,1,1,0,0,1,1,1,1,1,0,0,0,1,1,1,0,NaN
3,Arkansas,9893.750000,179.111945,1.478890,178.073690,92.120812,1,1,0,1,0,1,1,1,0,0,1,0,1,1,1,1,NaN
4,California,5570.920502,184.114804,1.949144,257.777251,174.228803,1,0,0,0,1,1,1,1,1,1,1,0,1,0,1,1,NaN
5,Colorado,5988.235294,156.123188,3.682579,283.083328,112.057771,1,1,0,1,1,1,1,1,1,0,1,0,1,0,1,1,NaN
6,Connecticut,56650.000000,144.561166,10.121205,101.311630,114.481865,1,1,0,1,1,1,1,1,1,1,1,1,1,0,1,0,NaN
7,Delaware,28671.428571,266.087388,7.571650,278.950659,180.529372,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,NaN
8,District of Columbia,12823.529412,212.744652,9.735756,94.774079,76.453934,1,1,0,0,1,1,1,1,1,0,1,1,1,0,1,0,NaN
9,Florida,20566.055046,95.906952,2.054686,260.825836,166.333412,1,1,0,1,0,1,1,1,1,1,0,0,1,1,1,1,NaN
